In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os


In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define augmentation settings
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values to [0,1]
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
save_dir = '/content/augmented-images'

# Create the save directory if it doesn't exist
os.makedirs(save_dir, exist_ok=True)

train_generator = train_datagen.flow_from_directory(
    "/content/dataset2",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/dataset2',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 2184 images belonging to 4 classes.
Found 2184 images belonging to 4 classes.


In [5]:
!cp -r "/content/drive/MyDrive/dataset_mod" "/content/dataset2"

In [6]:
model = tf.keras.models.load_model("/content/drive/MyDrive/Fire-master (2)/Fire-master/8thNov_2nd.h5")

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a checkpoint callback
checkpoint = ModelCheckpoint('best_model.h5',
                             monitor='val_loss',
                             save_best_only=True,
                             mode='min',
                             verbose=1)

In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')
i = 20
model.fit_generator(train_generator, epochs=i, validation_data=test_generator,
                   steps_per_epoch=len(train_generator),
                   validation_steps=len(test_generator))

<ipython-input-8-6ca4a4a8d093>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator, epochs=i, validation_data=test_generator,


Epoch 1/20
69/69 [==============================] - 91s 906ms/step - loss: 0.1599 - accuracy: 0.9524 - val_loss: 0.0863 - val_accuracy: 0.9766
Epoch 2/20
69/69 [==============================] - 64s 939ms/step - loss: 0.0854 - accuracy: 0.9707 - val_loss: 0.0769 - val_accuracy: 0.9840
Epoch 3/20
69/69 [==============================] - 63s 911ms/step - loss: 0.0668 - accuracy: 0.9785 - val_loss: 0.0466 - val_accuracy: 0.9876
Epoch 4/20
69/69 [==============================] - 62s 904ms/step - loss: 0.0705 - accuracy: 0.9789 - val_loss: 0.0962 - val_accuracy: 0.9771
Epoch 5/20
69/69 [==============================] - 65s 940ms/step - loss: 0.0473 - accuracy: 0.9853 - val_loss: 0.0398 - val_accuracy: 0.9886
Epoch 6/20
69/69 [==============================] - 62s 898ms/step - loss: 0.0304 - accuracy: 0.9895 - val_loss: 0.0557 - val_accuracy: 0.9867
Epoch 7/20
69/69 [==============================] - 62s 906ms/step - loss: 0.0394 - accuracy: 0.9876 - val_loss: 0.0456 - val_accuracy: 0.9872

In [13]:

import tensorflow as tf

In [15]:
model.save("fire_det.h5")

In [16]:
model_new = tf.keras.models.load_model("/content/fire_det.h5")

In [18]:
import tensorflow as tf

import os

def process_image(image_path, IMG_SIZE=224):

#     if not os.path.exists(image_path) or os.path.getsize(image_path) == 0:
# #         print(f"Invalid image file: {image_path}")
#         return

    image = tf.io.read_file(image_path)
    image = tf.io.decode_image(image, channels=3, expand_animations= False)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    return image
images_loc = []

In [19]:
images_loc = []
labels = []
for i in os.listdir("/content/dataset2/s0"):
    images_loc.append(process_image("/content/dataset2/s0"+"/"+i))
    labels.append(0)
for i in os.listdir("/content/dataset2/s1"):
    images_loc.append(process_image("/content/dataset2/s1"+"/"+i))
    labels.append(1)
for i in os.listdir("/content/dataset2/s2"):
    images_loc.append(process_image("/content/dataset2/s2"+"/"+i))
    labels.append(2)
for i in os.listdir("/content/dataset2/s3"):
    images_loc.append(process_image("/content/dataset2/s3"+"/"+i))
    labels.append(3)

In [20]:
import numpy as np
images_loc = np.array(images_loc)
labels = np.array(labels)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
X_train, X_test, y_train, y_test = train_test_split(images_loc, labels, test_size=0.3, random_state = 114)
y_preds = model_new.predict(X_test)
preds = y_preds.argmax(axis = 1)
print(f"Accuracy", accuracy_score(np.array(preds), np.array(y_test)))

21/21 [==============================] - 2s 53ms/step
Accuracy 0.9862804878048781
